# Tracer Advection Stencil

### Start a parallel cluster

In [ ]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines="mpi", n=6).start_and_connect_sync()
%autopx

### Input parameters

In [ ]:
%%capture
nx = 100
ny = 100
nhalo = 3
""" (int): number of points and halo points in each direction """
timestep = 900.0
""" (int): advection time step in seconds """
nDays = 12
""" (int): number of days to advect for """

test_case = "b"
""" (string): a: fortran test_case1; b: less dispersive """
print_advectionProgress = True
""" (bool): prints out a message every 100th timestep """
plot_outputDuring = False
""" (bool): plot output as advection is active (super slow!) """
plot_outputAfter = False
""" (bool): plot output after advection is complete (slow) """
plot_jupyterAnimation = True
""" (bool): plot an interactive animation after advection is done (slow) """
figure_everyNhours = 6
""" (int): number of model hours between figures """
write_initialCondition = True
""" (bool): creates a netCDF file with initial state """
plot_gridLayout = True
""" (bool): plots and saves grid layout to file grid_map.png """
show_figures = False
""" (bool): shows figures inside Jupyter notebook """


# initial condition adjustables
pressure_base = 10
""" (float): pressure of layer in Pa """
tracer_base = 1.0
""" (float): maximum tracer amount """
tracer_target_tile = 0
""" (int): rank of tile on which to center the tracer (0-5) """
density = 1
""" (float): air density """

#### Configured based on input parameters

In [ ]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import os

mpi_comm = MPI.COMM_WORLD

nSeconds = nDays * 86400 + timestep * 5
figure_everyNsteps = int(3600 / timestep * figure_everyNhours)
nSteps_advection = int(nSeconds / timestep + 1)

if figure_everyNsteps > nSteps_advection:
    figure_everyNsteps = nSteps_advection

pwd = os.getcwd()

namelistDict = func.store_namelist_variables(locals())

### Prepare everything for advection

In [ ]:
metadata, rank = func.define_metadata(namelistDict, mpi_comm)
tracer_dict = {
    "target_tile": tracer_target_tile,
    "rank": rank,
    "tracer_base": tracer_base,
}

configuration = func.configure_domain(mpi_comm, metadata["dimensions"])
if plot_gridLayout:
    func.plot_grid(configuration, metadata, rank, show=show_figures)

if test_case == "a":
    initialState = func.create_initialstate_testcase1a(
        configuration["grid_data"], metadata, tracer_dict, pressure_base
    )
elif test_case == "b":
    initialState = func.create_initialstate_testcase1b(
        configuration["grid_data"], metadata, tracer_dict, pressure_base
    )

tracAdv_data, tracAdv = func.prepare_everything_for_advection(
    configuration, initialState, metadata, density, timestep
)
tracAdv_dataInit = cp.deepcopy(tracAdv_data)

if write_initialCondition:
    fOut = pwd + "/data/" + "initialState_%sx%s_%s.nc" % (nx, ny, test_case)
    func.write_initial_condition_tofile(
        fOut, initialState, metadata, configuration, rank
    )

if plot_outputDuring or plot_outputAfter:
    plotDict_tracer = {
        "vmin": 0,
        "vmax": 1,
        "units": "",
        "title": "Tracer concentration",
        "cmap": "viridis",
    }

### Advection loop

In [ ]:
tracer_archive = [cp.deepcopy(tracAdv_data["tracers"]["tracer"])]
timestep_archive = [0]

for step in range(nSteps_advection):
    if (rank == 0) and (step + 1) % 100 == 0 and print_advectionProgress:
        print("Step: %d" % (step + 1))

    if plot_outputDuring and (step) % figure_everyNsteps == 0:
        tracer_global = configuration["communicator"].gather(
            tracAdv_data["tracers"]["tracer"]
        )
        fOut = (
            pwd
            + "/figs/"
            + "tracerAdvection_%s_%06.2f.png" % (test_case, (step * timestep) / 60 / 60)
        )
        plotDict_tracer["title"] = "Tracer state @ hour: %.2f" % (
            (step * timestep) / 60 / 60
        )
        func.plot_projection_field(
            configuration,
            metadata,
            tracer_global,
            plotDict_tracer,
            rank,
            fOut,
            show=show_figures,
        )

    tracAdv_data = func.run_advection_step_with_reset(
        tracAdv_dataInit, tracAdv_data, tracAdv, timestep
    )

    tracer_archive.append(cp.deepcopy(tracAdv_data["tracers"]["tracer"]))
    timestep_archive.append(step + 1)

In [ ]:
if plot_jupyterAnimation:
    func.plot_tracer_animation(
        configuration,
        metadata,
        tracer_archive,
        rank,
        plotDict_tracer,
        figure_everyNsteps,
        timestep,
    )

In [ ]:
if plot_outputAfter:
    for step in range(0, nSteps_advection, figure_everyNsteps):
        tracer_global = configuration["communicator"].gather(tracer_archive[step])
        fOut = (
            pwd
            + "/figs/"
            + "tracerAdvection_%s_%06.2f.png" % (test_case, (step * timestep) / 60 / 60)
        )
        plotDict_tracer["title"] = "Tracer state @ hour: %.2f" % (
            (step * timestep) / 60 / 60
        )
        func.plot_projection_field(
            configuration,
            metadata,
            tracer_global,
            plotDict_tracer,
            rank,
            fOut,
            show=show_figures,
        )

In [ ]:
%autopx

In [ ]:
cluster.shutdown()